In [1]:
import sys
from pathlib import Path

SRC = Path().resolve() / "src"
if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn

In [3]:
dataset_usage = 1
dataset_regenerate = False

seq_size = 16 # число токеном в скользящем окне датасета
seq_stride = 1

emb_dim=256
hidden_dim=512
num_layers=2
dropout_p=0.3
learning_rate=1e-3
weight_decay=0.01

model_train = True
model_load_path = "models/next_token_8_250820_184932.pth"

In [4]:
from data_utils import process_dataset, read_splits, truncate

if dataset_regenerate:
    process_dataset("data/raw_dataset.csv")

train_df, val_df, test_df = read_splits("data/dataset_processed.csv")

train_texts = truncate(list(train_df['text']), dataset_usage)
val_texsts = truncate(list(val_df['text']), dataset_usage)

print(len(train_texts))

1280000


In [5]:
from transformers import BertTokenizerFast
from tokenizer_utils import resolve_eos_token_id

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

eos_token_id = resolve_eos_token_id(tokenizer)

print("eos_token_id:", eos_token_id)

eos_token_id: 102


In [6]:

from torch.utils.data import DataLoader
from next_token_dataset import NextTokenDataset

train_dataset = NextTokenDataset(
    train_texts, 
    tokenizer, 
    eos_id=eos_token_id, 
    seq_size=seq_size, 
    stride=seq_stride
)

val_dataset = NextTokenDataset(
    val_texsts,
    tokenizer,
    eos_id=eos_token_id,
    seq_size=seq_size,
    stride=seq_stride
)

print(f"Train samples count: {len(train_dataset)}")
print(f"Val samples count: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

100%|██████████| 160000/160000 [00:05<00:00, 27084.21it/s]

Train samples count: 4158366
Val samples count: 517103


In [7]:
from device_utils import get_device
device = get_device()
print("Using device:", device)

Using device: mps


In [8]:
from lstm_model import NextTokenLSTM

model = NextTokenLSTM(
    vocab_size=tokenizer.vocab_size,
    emb_dim=emb_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout_p=dropout_p,
)

In [9]:
from lstm_train import train_next_token
from datetime import datetime

if model_train:
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    model = train_next_token(
        model,
        optimizer=optimizer,
        criterion=criterion,
        tokenizer=tokenizer,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        epochs=5
    )

    timestamp = datetime.now().strftime("%y%m%d_%H%M%S")
    torch.save(model.state_dict(), f"models/next_token_{seq_size}_{timestamp}.pth")
else:
    state = torch.load(model_load_path, map_location=device)
    model.load_state_dict(state)

Epoch: 1:  83%|████████▎ | 26918/32488 [33:28<06:55, 13.40it/s] 


KeyboardInterrupt: 

In [ ]:
from autocomplete import autocomplete_text

comp_text = autocomplete_text(
    model=model, 
    tokenizer=tokenizer,
    eos_id=eos_token_id,
    seq_size=seq_size,
    text="she is my enemy and",
)
comp_text

'i m not sure i m not sure i m going to be a long day i m not sure i m going to be a long day i m not sure i m going to bed i m so tired i m so tired i m so tired i'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

class GPT2Autocomplete:
    def __init__(self):
        # 1) Загрузка токенизатора и модели
        model_name = "distilgpt2"          # лёгкая версия GPT-2
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

        # 2) Создаём pipeline для генерации
        self.generator = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            device=0  # -1 = CPU; 0 = первый GPU (если есть)
        )

    def generate(self, prompt):
        out = self.generator(
            prompt,
            max_length=80,       # итоговая длина (включая prompt)
            num_return_sequences=1,
            do_sample=True,      # стохастическая генерация
            top_p=0.95,          # nucleus sampling
            temperature=0.8
        )

        return out[0]["generated_text"]
    
gpt_autocomplete = GPT2Autocomplete()

completition = gpt_autocomplete.generate("she is my")
completition

AttributeError: 'NextTokenLSTM' object has no attribute 'config'